# Тетрадка

тетрадка для подведения итогов расчета

In [ ]:
import os
import sys
sys.path.append('../../../')
import pandas as pd
import unifloc.uniflocpy.uTools.plotly_workflow as pw
import datetime
from multiprocessing import Pool

In [ ]:
import plot_workflow.plotly_option as pltl_opt
import plot_workflow.plotly_workflow as pltl_wf

from preproc_p import workflow_cs_data
from preproc_p import workflow_chess_data
from preproc_p import preproc_tool
from preproc_p import workflow_calc_data
from preproc_p import workflow_tr_data
from preproc_p import filtration
from proc_p import processor as proc

from ml import calibr_restore as calibr_restore
from postproc_p import result_and_metrics as result_and_metrics

In [ ]:
current_path = os.getcwd()
time_mark = '' #datetime.datetime.today().strftime('%Y_%m_%d_%H_%M_%S')
path_to_data = current_path + "\\data\\"
dirnames_list = []
for (dirpath, dirnames, filenames) in os.walk(path_to_data):
    dirnames_list.extend(dirnames)
    break
print(dirnames_list)

In [ ]:
calculated_wells = ['1354', '1479', '1509', '1540', '1567', '1602', '1628',
              '202', '252', '326', '353', '507', '540', '569', '570', '601',
              '627', '658', '689', '693']

In [ ]:
metrics = []
overall_data_list = []
for well_name in calculated_wells:
    
    path_to_work_dir = current_path + "\\data\\" + well_name +  "\\"
    path_to_adapt_dir = 'adaptation_' + '\\'
    path_to_restore_dir = 'restore_' + '\\'

    current_path = os.getcwd()
    path_to_data = current_path + "\\data\\"
    path_to_work_dir = current_path + "\\data\\" + well_name +  "\\"
    this_path = path_to_work_dir + path_to_restore_dir + well_name + '_adapt_restore_metrics_tr_report' +  '.csv'
    overall_metrics = pd.read_csv(this_path)
    metrics.append(overall_metrics)
    


In [ ]:
result = metrics[0]
for i in metrics[1:]:
    result = result.append(i)
    

In [ ]:
import datetime
mark = str(datetime.datetime.today()).replace(':','_')

In [ ]:
result.to_excel(f'result_{mark}.xlsx')